In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

Loading the 'customers' file:

In [2]:
customers = pd.read_csv('data/customers.csv')
events = pd.read_csv('data/events.csv')
offers = pd.read_csv('data/offers.csv')

In [3]:
print(customers.info())
print(customers.shape)
print(customers.describe)
print(customers.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       17000 non-null  object 
 1   became_member_on  17000 non-null  int64  
 2   gender            14825 non-null  object 
 3   age               17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB
None
(17000, 5)
<bound method NDFrame.describe of                             customer_id  became_member_on gender  age  \
0      68be06ca386d4c31939f3a4f0e3dd783          20170212    NaN  118   
1      0610b486422d4921ae7d2bf64640c50b          20170715      F   55   
2      38fe809add3b4fcf9315a9694bb96ff5          20180712    NaN  118   
3      78afa995795e4d85b5d9ceeca43f5fef          20170509      F   75   
4      a03223e636434f42ac4c3df47e8bac43          20170804    NaN  118   
...        

In [4]:
customers['gender'].unique()

array([nan, 'F', 'M', 'O'], dtype=object)

There are 3 categories M - Male, F - Female, and O - Others, so having null values in the 'gender' column does not make sense.

Creating filters to fiter out the null values:

In [5]:
non_null_gender = customers['gender'].notnull()

Checking the rows with null values in 'gender' column just to verify we aren't losing anything important:

In [6]:
null_gender = customers['gender'].isnull()
customer_gender_is_null = customers[null_gender]
customer_gender_is_null.sort_values(by= 'became_member_on', ascending = False).values.tolist()

[['75afb8d248164d62b326d4b1cc13303c', 20180726, nan, 118, nan],
 ['8f7ccdf48ce94663821995f157a16dd7', 20180725, nan, 118, nan],
 ['3cc127b1eafe4a9991d5bc02bab3bf77', 20180725, nan, 118, nan],
 ['d32aa738e03f44feac7ec3db9ec12baa', 20180723, nan, 118, nan],
 ['79a8c6c9f9e34117b8793c583ec16521', 20180722, nan, 118, nan],
 ['a770f6bc6b074a4da3c13a783dd7f4ca', 20180722, nan, 118, nan],
 ['30157b7fc37f43f2ae7ccd9edfe0b672', 20180722, nan, 118, nan],
 ['f8f7f2a0836e4ddf929e46a283f44b82', 20180721, nan, 118, nan],
 ['4719086c325c4591806f043fa2ca270a', 20180721, nan, 118, nan],
 ['a0661ae79ac14d6aaa6a512c33bb9dea', 20180721, nan, 118, nan],
 ['6c8127555baa4332a22044807c26aa04', 20180720, nan, 118, nan],
 ['0c96e5392702403b8db72f8781784abe', 20180720, nan, 118, nan],
 ['6e765b58b2414b1ea6617af14aa8c42f', 20180720, nan, 118, nan],
 ['12cca3942a6641428967693f1e90d3df', 20180720, nan, 118, nan],
 ['6c6fe42e0cf24caea7142531a84d15ca', 20180719, nan, 118, nan],
 ['a5519958fab34c1f97ed3c455160c5d6', 20

New dataframe after filtering out the null values:

In [7]:
customers = customers[non_null_gender]

Changing the datatype of column 'became_member_on' to string so that we can extract the date values into a date format of "year-month-day":

In [8]:
customers['became_member_on'] = customers['became_member_on'].astype(str)

extracting the date values and concatening them to the correct date format:

In [9]:
customers['became_member_on'] = customers['became_member_on'].str[:4] + '-' + customers['became_member_on'].str[4:6]+'-'+customers['became_member_on'].str[-2:]

Verifying the change has been successfully done:

In [10]:
customers.head()

,customer_id,became_member_on,gender,age,income
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0
5,e2127556f4f64592b11af22de27a7932,2018-04-26,M,68,70000.0
8,389bc3fa690240e798340f5a15918d5c,2018-02-09,M,65,53000.0
12,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,M,58,51000.0


In [11]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       14825 non-null  object 
 1   became_member_on  14825 non-null  object 
 2   gender            14825 non-null  object 
 3   age               14825 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 694.9+ KB


Finally, download the cleaned data as a new CSV file: 

In [12]:
customers.to_csv('outputs/customers_cleaned.csv')

Next, analyzing the 'events' file:

In [13]:
print(events.info())
print(events.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  306534 non-null  object
 1   event        306534 non-null  object
 2   value        306534 non-null  object
 3   time         306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB
None
                        customer_id           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0  
3  {'offer i

Checking what are the values in column 'event':

In [14]:
events['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

Extracting offer received and offer viewed events only and merging them into a separate new dataframe using an outer join to get all offers received or viewed:

In [15]:
event_OR_filter = events['event'] == 'offer received'
event_OV_filter = events['event'] == 'offer viewed'
event_OR = events[event_OR_filter]
event_OV = events[event_OV_filter]
merged_OROV = pd.merge(event_OR,event_OV, how= 'outer')

In [16]:
offers

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7,"['web', 'email', 'mobile', 'social']"
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,"['web', 'email', 'mobile', 'social']"
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,"['web', 'email', 'mobile']"


Extracting the offer id values and combining with 'offers' dataframe to create an extended version of the dataframe using an inner join. This new extended dataframe contains all offer received or viewed with offer type detail:

In [17]:
merged_OROV['value'] = merged_OROV['value'].str[14:-2]
merged_OROV.rename(columns = {'value' : 'offer_id'}, inplace = True)
offer_received_or_viewed_extended = pd.merge(merged_OROV, offers, on='offer_id', how='inner')

In [18]:
offer_received_or_viewed_extended['days'] = offer_received_or_viewed_extended['time']/24

Finally, downloading the modified data as a new CSV file: 

In [19]:
offer_received_or_viewed_extended.to_csv('outputs/offer_received_or_viewed_extended.csv')

Extracting only offers completed events:

In [20]:
event_OC_filter = events['event'] == 'offer completed'
event_OC = events[event_OC_filter]
event_OC['offer_id'] = event_OC['value'].str[14:-15]
event_OC['reward'] = event_OC['value'].str[-2]
event_OC['offer_id'] = event_OC['offer_id'].map(lambda x: x.rstrip("'"))
event_OC = event_OC.drop(columns = ['value'])

C:\Users\yania\AppData\Local\Temp\ipykernel_10832\1684524730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_OC['offer_id'] = event_OC['value'].str[14:-15]
C:\Users\yania\AppData\Local\Temp\ipykernel_10832\1684524730.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_OC['reward'] = event_OC['value'].str[-2]
C:\Users\yania\AppData\Local\Temp\ipykernel_10832\1684524730.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

verifying the extraction is successful:

In [21]:
print(event_OC.shape)
print(event_OC.info())
print(event_OC.head())

(33579, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 33579 entries, 12658 to 306527
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  33579 non-null  object
 1   event        33579 non-null  object
 2   time         33579 non-null  int64 
 3   offer_id     33579 non-null  object
 4   reward       33579 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB
None
                            customer_id            event  time  \
12658  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed     0   
12672  fe97aa22dd3e48c8b143116a8403dd52  offer completed     0   
12679  629fc02d56414d91bca360decdfa9288  offer completed     0   
12692  676506bad68e4161b9bbaffeb039626b  offer completed     0   
12697  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed     0   

                               offer_id reward  
12658  2906b810c7d4411798c6938adc9daaa5      2  
12672  fafdcd668e3743c1bb461111dcafc2a4     

In [22]:
event_OC.to_csv('outputs/offer_completed_only.csv')

Extracting only transaction events:

In [23]:
event_T = events[events['event'] == 'transaction']
event_T = event_T[event_T['value'].str.len() < 18]
event_T['value'] = event_T['value'].str[10:-1]
event_T.rename(columns = {'value' : 'Amount'}, inplace = True)
event_T['Amount']= event_T['Amount'].astype(float)
event_T['Amount']= event_T['Amount'].round(2)

In [24]:
event_T

,customer_id,event,Amount,time
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,34.56,0
12659,54890f68699049c2a04d415abc25e717,transaction,13.23,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,19.51,0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,18.97,0
12678,629fc02d56414d91bca360decdfa9288,transaction,33.90,0
...,...,...,...,...
306526,24f56b5e1849462093931b164eb803b5,transaction,22.64,714
306528,5ca2620962114246ab218fc648eb3934,transaction,2.20,714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,9.53,714
306531,a00058cf10334a308c68e7631c529907,transaction,3.61,714


Verifying extraction is successful:

In [25]:
print(event_T.shape)
print(event_T.head())
print(event_T.info())

(133038, 4)
                            customer_id        event  Amount  time
12657  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction   34.56     0
12659  54890f68699049c2a04d415abc25e717  transaction   13.23     0
12670  b2f1cd155b864803ad8334cdf13c4bd2  transaction   19.51     0
12671  fe97aa22dd3e48c8b143116a8403dd52  transaction   18.97     0
12678  629fc02d56414d91bca360decdfa9288  transaction   33.90     0
<class 'pandas.core.frame.DataFrame'>
Index: 133038 entries, 12657 to 306533
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   customer_id  133038 non-null  object 
 1   event        133038 non-null  object 
 2   Amount       133038 non-null  float64
 3   time         133038 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 5.1+ MB
None


In [26]:
event_T.to_csv('outputs/transaction_only.csv')

A transaction is considered for an offer only if the time of offer completed and transaction is same for a customer

Combining the offer completed and transaction events with an inner join to get all completed offer with associated transactions:

In [27]:
merged_OCT = pd.merge(event_OC, event_T, on=['customer_id', 'time'], how='inner')

This is the new dataframe that contains completed offers with related transaction based on same customer_id and time values:

In [28]:
merged_OCT.head()

,customer_id,event_x,time,offer_id,reward,event_y,Amount
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2,transaction,34.56
1,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,fafdcd668e3743c1bb461111dcafc2a4,2,transaction,18.97
2,629fc02d56414d91bca360decdfa9288,offer completed,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,transaction,33.90
3,676506bad68e4161b9bbaffeb039626b,offer completed,0,ae264e3637204a6fb9bb56bc8210ddfd,0,transaction,18.01
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,transaction,19.11


Verify the modifications have been made correctly:

In [29]:
print(merged_OCT.shape)
print(merged_OCT.info())
print(merged_OCT.describe())
print(merged_OCT.head())

(33141, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33141 entries, 0 to 33140
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  33141 non-null  object 
 1   event_x      33141 non-null  object 
 2   time         33141 non-null  int64  
 3   offer_id     33141 non-null  object 
 4   reward       33141 non-null  object 
 5   event_y      33141 non-null  object 
 6   Amount       33141 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.8+ MB
None
               time        Amount
count  33141.000000  33141.000000
mean     400.814701     18.084964
std      192.403249     16.146042
min        0.000000      0.150000
25%      240.000000     11.050000
50%      432.000000     16.720000
75%      564.000000     23.330000
max      714.000000    723.100000
                        customer_id          event_x  time  \
0  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed     0   
1  fe9

In [30]:
offers

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7,"['web', 'email', 'mobile', 'social']"
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,"['web', 'email', 'mobile', 'social']"
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,"['web', 'email', 'mobile']"


In [31]:
offers_completed_W_transaction_extended = pd.merge(merged_OCT, offers, on='offer_id', how='inner')

In [32]:
offers_completed_W_transaction_extended['days']= offers_completed_W_transaction_extended['time']/24

Finally, downloading the modified data as a new CSV file: 

In [33]:
offers_completed_W_transaction_extended.to_csv('outputs/offers_completed_W_transaction_extended.csv')